In [ ]:
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, Any

# Constants
FLASK_HOST: str = "127.0.0.1:5000"
FASTAPI_HOST: str = "127.0.0.1:8000"
USERS: list[int] = [100, 200, 300, 400, 500]  # Example user increments
SPAWN_RATE: int = 50
RUN_TIME: str = "30s"
RESULTS_DIR: str = "results"

# Ensure results directory exists
os.makedirs(RESULTS_DIR, exist_ok=True)


def run_test(api_name: str, host: str) -> Dict[str, list[float]]:
    """
    Runs tests for the given API and returns a dictionary with average and max response times.
    """
    avg_response_times: list[float] = []
    max_response_times: list[float] = []
    for users in USERS:
        print(f"Running test for {api_name} with {users} users...")
        result_file_prefix: str = f"{RESULTS_DIR}/{api_name}_{users}_users"
        cmd: str = (
            f"locust --host=http://{host} --headless --users={users} --spawn-rate={SPAWN_RATE} --run-time={RUN_TIME} --csv={result_file_prefix}"
        )
        subprocess.run(cmd, check=True, shell=True)
        result_file: str = f"{result_file_prefix}_stats.csv"
        df: pd.DataFrame = pd.read_csv(result_file)
        avg_response_times.append(df["Average Response Time (ms)"][0])
        max_response_times.append(df["Max Response Time (ms)"][0])

    return {
        "avg_response_times": avg_response_times,
        "max_response_times": max_response_times,
    }


def plot_results(results: Dict[str, Dict[str, list[float]]]) -> None:
    """
    Plots the results of the API tests.
    """
    plt.figure(figsize=(12, 6))
    for i, (api_name, times) in enumerate(results.items(), start=1):
        plt.subplot(1, 2, i)
        plt.plot(
            USERS,
            times["avg_response_times"],
            marker="o",
            linestyle="-",
            color="b",
            label="Average Response Time",
        )
        plt.plot(
            USERS,
            times["max_response_times"],
            marker="o",
            linestyle="-",
            color="r",
            label="Max Response Time",
        )
        plt.title(f"Response Times - {api_name}")
        plt.xlabel("Number of Users")
        plt.ylabel("Response Time (ms)")
        plt.legend()
    plt.tight_layout()
    plt.show()




In [ ]:
results: Dict[str, Dict[str, Any]] = {
    "Flask": run_test("flask", FLASK_HOST),
    "FastAPI": run_test("fastapi", FASTAPI_HOST),
}


In [ ]:
plot_results(results)